In [ ]:
! pip install -q ragas>=0.2.0 langchain-community langchain-openai langchain_core
! pip install -q openai rapidfuzz

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.3 MB/s eta 0:00:00


# 1. Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import json
import warnings
import pandas as pd
import random
import rapidfuzz

from pathlib import Path
from typing import List, Optional
from google.colab import userdata
from openai import OpenAI
from ragas.embeddings import HuggingfaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.cache import DiskCacheBackend
from ragas.llms import llm_factory
from ragas.testset import TestsetGenerator
from langchain_core.documents import Document

warnings.filterwarnings('ignore')

# 2. Load chunks

In [ ]:
def load_chunks_flexible(path: str, default_source: str | None = None):
    p = Path(path)
    text = p.read_text(encoding="utf-8").replace("\r\n", "\n")  # ăn cả CRLF/LF

    # Bắt từng CHUNK block
    chunk_pat = re.compile(
        r"^===== CHUNK (?P<id>\d+) =====\s*\n(?P<body>.*?)(?=^===== CHUNK|\Z)",
        re.S | re.M,
    )

    docs = []
    for m in chunk_pat.finditer(text):
        chunk_id = int(m.group("id"))
        body = m.group("body").strip()

        # Thử parse metadata nếu có Source/Page ở đầu body
        meta_pat = re.compile(
            r"^Source:\s*(?P<source>.*?)\s*\n"
            r"Page:\s*(?P<page>.*?)\s*\n\s*\n"
            r"(?P<content>.*)$",
            re.S,
        )
        mm = meta_pat.match(body)

        if mm:
            source = mm.group("source").strip()
            page_raw = mm.group("page").strip()
            content = mm.group("content").strip()
            page = int(page_raw) if page_raw.isdigit() else None
        else:
            source = default_source or p.name
            page = None
            content = body

        if not content:
            continue

        docs.append(Document(
            page_content=content,
            metadata={"chunk_id": chunk_id, "source": source, "page": page},
            id=f"{source}::chunk{chunk_id}",
        ))

    return docs


# chunk_text_law = load_chunks_flexible("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/chunk_text_law.txt")
chunk_text_law = load_chunks_flexible("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_text_only_chunks1.txt")
print(len(chunk_text_law))

k = 10
samples = random.sample(chunk_text_law, k=min(k, len(chunk_text_law)))


282


# 2. RAGAS

In [ ]:
raw = userdata.get("RAG")
key = raw.strip()
os.environ["OPENAI_API_KEY"] = key

In [ ]:
# 2) OpenAI client + LLM (gpt-4o-mini) + cache
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
generator_llm = llm_factory(
    "gpt-4o-mini",
    client=client,
    temperature=0,          # để cache hit tốt hơn (nên giữ cố định)
)

# 3) Embedding: SentenceTransformer model (Vietnamese_Embedding) + cache
# Docs: HuggingfaceEmbeddings dùng SentenceTransformer bên trong :contentReference[oaicite:4]{index=4}
lc_embeddings = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

generator_embeddings = LangchainEmbeddingsWrapper(lc_embeddings)

# 4) TestsetGenerator (chuẩn docs pre-chunked)
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
)

# 5) Bạn đã có chunks sẵn (List[Document]) -> generate_with_chunks (chuẩn docs)
testset = generator.generate_with_chunks(
    chunks=samples,   # <-- list Document của bạn
    testset_size=10,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

...

Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
out_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/testset/testset_qa.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)  # ✅ tạo thư mục nếu chưa có

testset.to_csv(str(out_path))
print(testset.to_pandas().head())


                                          user_input  \
0  What types of vehicles can a person with a D2 ...   
1  Trong trường hợp nào người điều khiển xe mô tô...   
2  Trong trường hợp gặp nạn nhân bị bỏng trong vụ...   
3  Người lái xe có được phép vượt xe trên cầu hẹp...   
4  Loại xe nào được phép tham gia giao thông với ...   

                                  reference_contexts  \
0  [id: Q131\nCâu 131. Người có Giấy phép lái xe ...   
1  [id: Q035\nCâu 35. Người điều khiển xe mô tô p...   
2  [id: Q205\nCâu 205. Khi gặp nạn nhân bị bỏng t...   
3  [id: Q047\nCâu 47. Người lái xe được phép vượt...   
4  [id: Q147\nCâu 147. Trên đường bộ (trừ đường c...   

                                           reference  \
0  A person with a D2 driver's license can operat...   
1  Người điều khiển xe mô tô phải giảm tốc độ hoặ...   
2  Người lái xe cần thực hiện các bước sau: Gọi s...   
3  Không được phép vượt xe trên cầu hẹp có một là...   
4  Ô tô tải, ô tô chở người trên 28 chỗ không 